<a href="https://colab.research.google.com/github/ss1705/ai-traffic-system/blob/main/Chain_of_Thought_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate torch sentencepiece --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.1 MB/s eta 0:00:00


In [2]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.1 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
from google.colab import userdata
hf_token = userdata.get('HFTOKEN')

In [5]:
from huggingface_hub import login
login(token=hf_token)

In [6]:
import torch

In [7]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=True,  # Quantized model to save RAM
    device_map="auto"  # Uses GPU efficiently
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [11]:
df = pd.read_csv('/content/drive/MyDrive/TrafficLLM/congestion_data.csv')

In [12]:
df = df.drop(["Description", "Street", "City", "County", "State", "Country", "ZipCode", "LocalTimeZone",
               "WeatherStation_AirportCode", "WeatherTimeStamp","ID"], axis=1, errors="ignore")

In [14]:
df["StartTime"] = pd.to_datetime(df["StartTime"], utc=True, errors="coerce")
df["EndTime"] = pd.to_datetime(df["EndTime"], utc=True, errors="coerce")

In [15]:
df = df.dropna(subset=["StartTime", "EndTime"])

In [16]:
df["StartTime"] = (df["StartTime"] - df["StartTime"].min()).dt.total_seconds()
df["EndTime"] = (df["EndTime"] - df["EndTime"].min()).dt.total_seconds()

In [17]:
df.head()

,Severity,Start_Lat,Start_Lng,StartTime,EndTime,Distance(mi),DelayFromTypicalTraffic(mins),DelayFromFreeFlowSpeed(mins),Congestion_Speed,Temperature(F),WindChill(F),Humidity(%),Pressure(in),Visibility(mi),WindDir,WindSpeed(mph),Precipitation(in),Weather_Event,Weather_Conditions
0,2,39.191032,-120.819740,27350400.0,27349070.0,1.40,2.58,2.60,Moderate,54.0,NaN,30.0,30.15,10.0,North,3.5,NaN,NaN,Clear
1,0,41.736015,-87.721565,87477540.0,87478351.0,0.73,0.42,1.00,Slow,39.0,31.8,70.0,30.06,10.0,West,11.5,NaN,NaN,Overcast
2,0,32.519043,-93.741096,158769180.0,158769935.0,1.80,1.00,2.00,Moderate,30.0,30.0,79.0,30.20,10.0,WNW,3.0,0.00,NaN,Fair
3,0,40.730564,-74.001709,150343620.0,150343945.0,1.42,1.00,2.00,Slow,48.0,48.0,93.0,29.92,1.0,VAR,3.0,0.04,NaN,Light Rain
4,1,33.758331,-118.238533,48653700.0,48656242.0,2.60,4.90,6.92,Slow,66.9,NaN,79.0,29.89,9.0,Calm,NaN,NaN,NaN,Overcast


In [65]:
sample_record = df.iloc[1]

In [66]:
print(sample_record)

Severity                                  0
Start_Lat                         41.736015
Start_Lng                        -87.721565
StartTime                        87477540.0
EndTime                          87478351.0
Distance(mi)                           0.73
DelayFromTypicalTraffic(mins)          0.42
DelayFromFreeFlowSpeed(mins)            1.0
Congestion_Speed                       Slow
Temperature(F)                         39.0
WindChill(F)                           31.8
Humidity(%)                            70.0
Pressure(in)                          30.06
Visibility(mi)                         10.0
WindDir                                West
WindSpeed(mph)                         11.5
Precipitation(in)                       NaN
Weather_Event                           NaN
Weather_Conditions                 Overcast
Name: 1, dtype: object


In [67]:
weather_condition = sample_record["Weather_Conditions"]
weather_event = sample_record["Weather_Event"]
temperature = sample_record["Temperature(F)"]
wind_speed = sample_record["WindSpeed(mph)"]
wind_direction = sample_record["WindDir"]
visibility = sample_record["Visibility(mi)"]
precipitation = sample_record["Precipitation(in)"] if not pd.isna(sample_record["Precipitation(in)"]) else 0.0
delay_pred = sample_record["DelayFromTypicalTraffic(mins)"]
congestion_pred = sample_record["Congestion_Speed"]

In [74]:
prompt = f"""
You are an expert in traffic analysis and prediction. Given the following traffic data, predict the **expected delay** and **congestion level**.

### **Traffic & Weather Data**
- **Weather Conditions:** {weather_condition} (Event: {weather_event})
- **Temperature:** {temperature}°F
- **Wind Speed:** {wind_speed} mph
- **Wind Direction:** {wind_direction}
- **Visibility:** {visibility} miles
- **Precipitation:** {precipitation} inches

### **Historical Trends (Similar Cases)**
From past data, similar weather conditions resulted in:
- **Case 1:** Delay: 6 mins, Congestion: Moderate
- **Case 2:** Delay: 12 mins, Congestion: Slow
- **Case 3:** Delay: 3 mins, Congestion: Fast

Use these examples to **make an accurate prediction based on this case.**

### **Prediction Task**
1️⃣ **Predict Delay:** Estimate the expected delay in minutes.
2️⃣ **Predict Congestion Level:** Categorize congestion as **Fast, Moderate, or Slow**.

### **Step-by-Step Reasoning**
1️⃣ **Compare to Past Cases:** How does this situation compare to the provided historical cases?
2️⃣ **Assess Weather Impact:** How do the given weather conditions (temperature, precipitation, wind, visibility) affect traffic?
3️⃣ **Analyze Road & Traffic Factors:** How do non-weather factors (e.g., road infrastructure, time of day) contribute to delays?
4️⃣ **Derive Predictions:** Based on the above points, what are the expected delay and congestion level?
5️⃣ **Suggest Solutions:** What traffic management strategies (e.g., rerouting, speed adjustments, public transport promotion) can help mitigate congestion?

### **Expected Output Format**
Please provide the answer in the following structured format:

**Predicted Delay:** X minutes
**Predicted Congestion Level:** Fast / Moderate / Slow
"""

In [75]:
print(prompt)


You are an expert in traffic analysis and prediction. Given the following traffic data, predict the **expected delay** and **congestion level**.

### **Traffic & Weather Data**
- **Weather Conditions:** Overcast (Event: nan)
- **Temperature:** 39.0°F
- **Wind Speed:** 11.5 mph
- **Wind Direction:** West
- **Visibility:** 10.0 miles
- **Precipitation:** 0.0 inches

### **Historical Trends (Similar Cases)**
From past data, similar weather conditions resulted in:
- **Case 1:** Delay: 6 mins, Congestion: Moderate  
- **Case 2:** Delay: 12 mins, Congestion: Slow  
- **Case 3:** Delay: 3 mins, Congestion: Fast  

Use these examples to **make an accurate prediction based on this case.**

### **Prediction Task**
1️⃣ **Predict Delay:** Estimate the expected delay in minutes.  
2️⃣ **Predict Congestion Level:** Categorize congestion as **Fast, Moderate, or Slow**.  

### **Step-by-Step Reasoning**
1️⃣ **Compare to Past Cases:** How does this situation compare to the provided historical cases?  

In [56]:
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [76]:
response = llm_pipeline(prompt, max_new_tokens=1000, return_full_text=False)[0]["generated_text"]
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



### **Example Output**

**Predicted Delay:** 6 minutes  
**Predicted Congestion Level:** Moderate  

### **Solution**
1️⃣ **Compare to Past Cases:** 
- Case 1: Delay of 6 minutes, Moderate congestion.
- Case 2: Delay of 12 minutes, Slow congestion.
- Case 3: Delay of 3 minutes, Fast congestion.

From these cases, we can see that the current weather conditions (overcast) are similar to Case 1, which resulted in a moderate delay and congestion level. However, the temperature is lower than in Case 1 (39.0°F vs 60.0°F), which could potentially lead to slower traffic due to icy road conditions. The wind speed and direction are also different from Case 1, which may affect traffic flow.

2️⃣ **Assess Weather Impact:**
- Temperature: Lower temperature could slow down traffic due to icy road conditions.
- Precipitation: No precipitation reported, so no direct impact on traffic.
- Wind: West wind could potentially cause delays if it blows against the direction of traffic flow.
- Visibility: Goo